<div style='float: right'><img src='pic/inshi.png'/></div>

## <div id='inshi' />因子の部屋

In [ ]:
#!pip3 install pulp ortoolpy
import pandas as pd
from math import log
from pulp import LpProblem, lpSum, lpDot, value
from ortoolpy import addbinvars
data = """\
ABBCD
AEEFD
GGHFD
IJHKK
ILHMM""".splitlines()
nums = {'A':6, 'B':15, 'C':1, 'D':12, 'E':20, 'F':8,
    'G':10, 'H':6, 'I':4, 'J':4, 'K':15, 'L':1, 'M':10}

### 問題
* すべてのマスに１からNまでの数字のどれかを１つずつ入ります（０は使いません）
* タテ列、ヨコ列のどれにも、１からNまでの数字が１つずつ入ります
* 太線で囲まれた四角形（部屋）の左上のマスに小さく書かれている数は、その部屋の各マスに入る数をすべてかけあわせた値となります

### 変数
* v：各マスでその数字かどうか (1)
* r：マスの数字 (2)

### 制約
* $v_{ij}$は1つの数字のみ (3)
* rをvで表す (4)
* 縦および横で同じ数字はない (5)
* 積が指定した数字 (6)

In [ ]:
nn = len(data)
a = pd.DataFrame([(i,j,k,data[i][j]) for i in range(nn)
    for j in range(nn) for k in range(1,nn+1)], columns=list('行列数字'))
a['Var'] = addbinvars(len(a))
a[:2]

In [ ]:
m = LpProblem()
for cl in [['行','列'],['行','数'],['列','数']]:
    for _,v in a.groupby(cl):
        m += lpSum(v.Var) == 1
for g,v in a.groupby('字'):
    e = lpDot([log(i) for i in v.数], v.Var) - log(nums[g])
    m += e >= -0.0001
    m += e <=  0.0001
m.solve()

In [ ]:
a['Val'] = a.Var.apply(value)
print(a[a.Val>0.5].数.values.reshape(nn,nn))